# Test of phonopy API on python

In [29]:
# Any constants copied over for easy transfer of code
PHONOPY_NUM_LINE_INTS = 50
PH_BAND_RAW_DATA_NAME = 'ph_dos_raw'

In [30]:
# Test box
from pymatgen.io.vasp.inputs import Kpoints

kpts_line = Kpoints.from_file('LINE_KPOINTS')
print(kpts_line.labels)

['\\Gamma', 'M', 'M', 'K', 'K', '\\Gamma']


In [31]:
import matplotlib.pyplot as plt
%matplotlib inline

import phonopy
from phonopy.phonon.band_structure import get_band_qpoints_and_path_connections
import numpy as np
PH_FORCE_SETS_NAME = "FORCE_SETS"
DIR_PHONOPY = './'
PH_BASIC_INFO_FILENAME = 'phonopy_basic_structure_input.txt'

# Load in the info from preprocessing and vasp force calculations
def ph_load_raw_data(dirName):
    # Since FORCE_SETS already exists we just need to give it SPOSCAR so it knows about the supercell
    #TODO checkpath
    try:
        print('Loading phonon raw data from FORCE_SETS...')
        phonon = phonopy.load(supercell_filename=dirName + "SPOSCAR", log_level=1)
        print('Load complete. Proceeding to analysis...')
        return phonon
    except Exception as err:
        print('Error in loading phonopy data: ' + err)
        sys.exit('Suggested source of error: you have either missing or invalid FORCE_SETS')

In [32]:
# Print basic info we might want to have
def ph_print_basic_info(phonon_obj, outDirName):
    #TODO checkpath
    try:
        f = open(PH_BASIC_INFO_FILENAME, 'w')
        f.write(str(phonon_obj.primitive))
        f.close()
        return
    except Exception as err:
        print('Error in writing basic info out to %s in directory %s.'%(PH_BASIC_INFO_FILENAME, DIR_PHONOPY))
        sys.exit('Error message:', err)

In [33]:
# Get the path for phonopy from LINE_KPOINTS
def ph_get_band_path(kpoints_line_obj):
    # Fancy way of removing duplicate rows, which VASP needs but phonopy does not want
    # NOTE: this function doesn't work well for complicated paths, but 2D materials are probably fine. See github's homepage README for more info.
    new_array = kpoints_line_obj.kpts
    np_path = np.unique(new_array, axis=0)
    path = []
    for i in np_path:
        path.append(list(i))
    path.append(path[0]) # We do want the last row to be the same as the first row, to get a closed path!
    return path

# Get the labels for the path
def ph_get_band_path_labels(kpoints_line_obj):
    # We really just need to get rid of any labels that don't 
    # NOTE: this function doesn't work well for complicated paths, but 2D materials are probably fine. See github's homepage README for more info.
    labelArr = list(dict.fromkeys(kpoints_line_obj.labels))
    gammaIndex = None

    # phonopy expects a special input style for Gamma different from vasp
    for i in range(0, len(labelArr)):
        if labelArr[i] == '\\Gamma':
            gammaIndex = i
    labelArr[gammaIndex] = '$\\Gamma$'

    # like with get_band_path, we append to the end the first label to close the path
    labelArr.append(labelArr[0])

    return labelArr

In [34]:
# Here we can get the band structure using the phonon object loaded in the load function
def get_band_structure(phonon, kpoints_line_obj, outDir, num_inters=PHONOPY_NUM_LINE_INTS, extract_raw_data=True, extract_plot=True):
    # TODO: outDir = checkPath(outDir)
    path = ph_get_band_path(kpoints_line_obj)
    labels = ph_get_band_path_labels(kpoints_line_obj)
    print('paths:', path)
    print('labels:', labels)
    qpoints, connections = get_band_qpoints_and_path_connections(path, npoints=num_inters)
    phonon.run_band_structure(qpoints, path_connections=connections, labels=labels)

    if extract_raw_data:
        try:
            f = open(outDir + PH_BAND_RAW_DATA_NAME, 'w')
            f.write(str(phonon.get_band_structure_dict()))
            f.close()
        except Exception as err:
            print('Error writing band structure raw info to file: ' + err)

    if extract_plot:
        phonon.plot_band_structure().show()
        # phonon.plot_band_structure().savefig(outDir)
    
    return

In [35]:
phonon = ph_load_raw_data(DIR_PHONOPY) # Get phonopy data object containing the info to parse for band/dos
ph_print_basic_info(phonon, DIR_PHONOPY) # Basic info about the crystal
get_band_structure(phonon, kpts_line, DIR_PHONOPY)

Loading phonon raw data from FORCE_SETS...
Supercell structure was read from "./SPOSCAR".
Force sets were read from "FORCE_SETS".
Max drift of force constants: -0.000008 (yy) -0.000004 (yy)
Max drift after symmetrization by translation: 0.000000 (yy) 0.000000 (yy)
Force constants were symmetrized.
Load complete. Proceeding to analysis...
paths: [[0.0, 0.0, 0.0], [0.5, 0.5, 0.0], [0.666667, 0.333333, 0.0], [0.0, 0.0, 0.0]]
labels: ['$\\Gamma$', 'M', 'K', '$\\Gamma$']


RuntimeError: Failed to process string with tex because latex could not be found